In [1]:
from bs4 import BeautifulSoup as bs

import json
import re
import os
import time
import random
import datetime
import calendar
import logging

import urllib
import urllib.request
from urllib.error import HTTPError

import sklearn
import matplotlib

import numpy as np
import pandas as pd
import sklearn.preprocessing as prep



import requests
import html.parser
from requests.exceptions import HTTPError
from socket import error as SocketError
from http.cookiejar import CookieJar

import feedparser
import pickle

import langdetect

In [3]:
from textstat.textstat import textstat as ts

In [190]:
from profanity import profanity 

In [273]:
from collections import Counter

In [191]:
profanity.contains_profanity('fuck')

True

In [32]:
### safely open the saved pickle file
def safelyOpen(nameOfSavedDictionary):
    try:
        with open(nameOfSavedDictionary + '.pickle', 'rb') as handle:
            return pickle.load(handle)
    except FileNotFoundError:
        print("File was not found ")

In [5]:
def saveToFile(stuffToSave, stuffNameOnDisc):
    if (stuffNameOnDisc + '.pickle') in os.listdir():
        print('NOTICE')
        print('the same named file found on disc, be super that you are not rewritting it in good way')
    with open(stuffNameOnDisc + '.pickle', 'wb') as handle:
        pickle.dump(stuffToSave, handle)

In [ ]:
## lets first clean the fake data

In [3]:
print(os.listdir())

['.DS_Store', '.idea', '.ipynb_checkpoints', '__pycache__', 'dirty_data_code_NYT.py', 'fake.csv', 'FakeData.ipynb', 'FakeData.py', 'fakeDictionaryTrialNUMBERSTrue.pickle', 'fakeKaggleEnglishOver300.pickle', 'getting_data.ipynb', 'kucapaca.pickle', 'LargerNytDataSetQuery.ipynb', 'newDict.pickle', 'NLPanalysis.ipynb', 'NLPanalysis.py', 'numericalFakeFromKaggle.pickle', 'numericalTrue2016NYT.pickle', 'nyt.csv', 'nyt.jsongetTimesArticles_testing.log', 'NytData.py', 'nytRawTestText2016.pickle', 'nytRawText2015.pickle', 'nytRawText2016.pickle', 'nytSnippets2015.pickle', 'nytSnippets2016.pickle', 'PutDataIntoSQL.ipynb', 'PutDataIntoSQL.py', 'testDic.pickle', 'TrainAndPredict.ipynb', 'TrainAndPredict.py', 'trial.pickle', 'trialList.pickle', 'trialList1.pickle', 'trueDictionaryTrialNUMBERSFalse.pickle']


In [4]:
dataFake = safelyOpen('fakeKaggleEnglishOver300')

In [8]:
def cleanText(text, classOfNews):
    if classOfNews == 'fake':
        text = text.replace('\n', ' ')
        text = text.split('.')
        del(text[-1])
        text.append('')
        text = '.'.join(text)
        return text
    elif classOfNews == 'notFake':
        pass
    else:
        print('wrong news class. Only \'fake\' of \'notFake\' are allowed ')
    

In [9]:
## calculate readability of the text

In [10]:
## calculate readability
def readability_of_text(text, score="dale_chall"):
    try:
        if type(score) == str:
            if score == "dale_chall":
                readability = ts.dale_chall_readability_score(text)
                return readability
                #print(readability)
            else:
                print('Other scores are not supported yet. You wanted: ' + score + " we have only dale_chall")
        else:
            raise ValueError
    except ValueError:
        print("the score shuld be of type str. You put " + str(type(score)))
        raise


In [131]:
# readability_of_text(cleanEx2)

In [113]:
nytData = safelyOpen('nytRawText2016')


In [14]:
textNYT = nytData['56ab384c38f0d83460bc7abf']


In [15]:
textNYT2 = nytData['56d31cea38f0d85dd0d93be2']

In [16]:
readability_of_text(textNYT)

8.37

In [17]:
readability_of_text(textNYT2)

9.09

In [18]:
nytDataPanda = pd.DataFrame.from_dict(nytData,'index',str)

In [119]:
nytDataPanda.index.name = 'article_index'

In [19]:
## OK I made a decision I will do the analysis in dictionary for final fit I will convert it to panda Data frame
## fake and not fake data MUST have the following INPUT structure dic = {article_id:text}
## I will produce new dictionary with values that MUST have the following structure:
## divVal = {article_id:[numbers,num,..]}

In [9]:
## function that converts the particular panda data frame to dictionary
## this function is Fake News from Kaggle-source specific
def createDictionary(pandaData, columnName):
    ## subsetting the pandaData
    ddt = pandaData.ix[:,str(columnName)]
    ## creating the dictionary
    return ddt.to_dict()
    

In [21]:
## this function cleans the text of the fake news a bit
def cleanTextInFakeDictionary(dicFake):
    dicFake = {new_key: cleanText(dicFake[new_key],'fake') for new_key in dicFake.keys()}
    return dicFake

In [183]:
## text examples

In [10]:
dicFake =  createDictionary(dataFake, 'text')

In [24]:
dicFake = cleanTextInFakeDictionary(dicFake)

In [85]:
textFake = dicFake['0206b54719c7e241ffe0ad4315b808290dbe6c0f']

In [90]:
textTrue = dicTrue['56aee62738f0d82f06c0cd1d']

In [29]:
## PIPELINE: turn FAKE panda data frame to required type of cleaned dictionary

In [30]:
dicFake = createDictionary(dataFake, 'text')

In [31]:
dicFake = cleanTextInFakeDictionary(dicFake)

In [176]:
## NYT data

In [5]:
dicTrue = safelyOpen('nytRawText2016')

In [33]:
## Create empty dictionaries to be our final dictionaries of the type: {'key':[]}

In [32]:
def createEmptyDictionary(keys):
    return {key: [] for key in keys}

In [34]:
## Lets Ccalculate the readabilities of the data

In [35]:
def calculateFunResults(fun, dictionary, finalDic):
    for key in dictionary.keys():
        finalDic[key].append(fun(dictionary[key]))
    return finalDic
    

In [136]:
def funnyName(fun):
    return str(fun).split(' ')[1]

In [135]:
funny(readability_of_text)

'readability_of_text'

In [233]:
numericalTrue = createEmptyDictionary(dicTrue.keys())

In [234]:
numericalTrue = calculateFunResults(readability_of_text, dicTrue, numericalTrue)

In [237]:
numericalFake = createEmptyDictionary(dicFake.keys())

In [238]:
numericalFake = calculateFunResults(readability_of_text, dicFake, numericalFake)

In [ ]:
print(di)

In [240]:
## save numericalFake and also the numericalTrue to file 

In [244]:
saveToFile(numericalFake, 'numericalFakeFromKaggle')

In [245]:
saveToFile(numericalTrue, 'numericalTrue2016NYT')

In [36]:
numericalFake = safelyOpen('numericalFakeFromKaggle')

In [37]:
numericalTrue = safelyOpen('numericalTrue2016NYT')

In [98]:
functionsAppliedInOrderToFakeAndTrue = ['readability']

In [100]:
functionsAppliedInOrderToFakeAndTrue.append('checkUpper')

In [101]:
functionsAppliedInOrderToFakeAndTrue

['readability', 'checkUpper']

In [252]:
## NOTE: do not forget to implement the list of names of functions that are in numericalTrue etc.

In [67]:
import textblob.tokenizers  as tt

In [75]:
words = tt.word_tokenize(textFake)

In [123]:
def checkUpperDensity(text):
    finalNumberOfUpper = 0
    length = 0.
    for word in tt.word_tokenize(text):
        length +=1.0
        if word.isupper():
            finalNumberOfUpper += 1
    return finalNumberOfUpper/length

In [124]:
checkUpperDensity(textFake)

0.019444444444444445

In [155]:
klun = list(tt.word_tokenize("hura hura hura"))

In [156]:
for i in klun:
    print(i)



hura
hura
hura


In [157]:
for i in klun:
    print(i)

hura
hura
hura


In [164]:
def eat(fun):
    return fun('TTT')

In [166]:
eat(checkUpper.__call__)

1

In [171]:
listOfFun = [checkUpper, checkLower]

In [172]:
for xxx in listOfFun:
    print(eat(xxx))

1
0


In [167]:
def checkLower(text):
    finalNumberOfLower = 0 
    for word in tt.word_tokenize(text):
        if not word.isupper():
            finalNumberOfLower += 1
    return finalNumberOfLower

In [163]:
checkUpper.__call__

<method-wrapper '__call__' of function object at 0x109e61598>

In [96]:
def checkUpper(text):
    finalNumberOfUpper = 0 
    for word in tt.word_tokenize(text):
        if word.isupper():
            finalNumberOfUpper += 1
    return finalNumberOfUpper

In [97]:
checkUpper(textFake)

7

In [104]:
## calculating the number of UPPER CASE WORDS in true set

In [105]:
numericalTrue = calculateFunResults(checkUpper, dicTrue, numericalTrue)

In [35]:
#numericalTrue

In [106]:
## calculating the number of UPPER CASE WORDS in false set

In [107]:
numericalFake = calculateFunResults(checkUpper, dicFake, numericalFake)

In [33]:
#numericalFake

In [187]:
## calculating the profanity levels of fake texts

In [194]:
numericalFake = calculateFunResults(vulgar, dicFake, numericalFake)

In [34]:
#numericalFake

In [39]:
from textblob import TextBlob

In [51]:
from textblob.sentiments import NaiveBayesAnalyzer

In [216]:
blob = TextBlob(textTrue)

In [59]:
blobFake = TextBlob(textFake)

In [219]:
blobFake.sentiment[1]

0.4487839221710191

In [56]:
from nltk.tokenize import TabTokenizer

In [192]:
def vulgar(text):
    prof = 0
    for word in tt.word_tokenize(text):
        if profanity.contains_profanity(word):
            prof += 1
    return prof

In [179]:
lstOfFun = [checkUpper, checkLower]

In [181]:
[fun.__name__ for fun in lstOfFun]

['checkUpper', 'checkLower']

In [180]:
lstOfFun

[<function __main__.checkUpper>, <function __main__.checkLower>]

In [175]:
listOfFunc = [checkLower.__name__, checkUpper.__name__]

In [220]:
### checking up the number of ? and !

In [ ]:
def calculateQuestAndExAndDensity(text):
    finalNumberOfUpper = 0
    length = 0.
    for word in tt.word_tokenize(text):
        length +=1.0
        if word.isupper():
            finalNumberOfUpper += 1
    return finalNumberOfUpper/length

In [223]:
textList = list(tt.word_tokenize(textFake))

In [244]:
temp = list(filter(lambda x: re.match('\?|!', x), '?!'))

In [245]:
temp

['?', '!']

In [215]:
###-----------###-----------###-----------###-----------###-----------###-----------###-----------###-----------

In [272]:
### OK LETS SMALLER DOWN THE fake news set

In [11]:
dicFakeSmaller = {key: dicFake[key] for key in list(dicFake.keys())[0:1000]}

In [12]:
dicTrueSmaller = dicTrue

In [13]:
lenGoldenFake = int(0.2*len(dicFakeSmaller))
lenGoldenTrue = int(0.2*len(dicTrueSmaller))

In [257]:
## creating the golden sets

In [14]:
dicFakeSmallerGOLDKeys = list(dicFakeSmaller.keys())[0:lenGoldenFake]
dicFakeSmallerCALCKeys = list(dicFakeSmaller.keys())[lenGoldenFake:len(dicFakeSmaller)]

In [15]:
random.shuffle(dicFakeSmallerGOLDKeys)
random.shuffle(dicFakeSmallerCALCKeys)

In [16]:
dicTrueSmallerGOLDKeys = list(dicTrueSmaller.keys())[0:lenGoldenTrue]
dicTrueSmallerCALCKeys = list(dicTrueSmaller.keys())[lenGoldenTrue:len(dicTrueSmaller)]

In [17]:
random.shuffle(dicTrueSmallerGOLDKeys)
random.shuffle(dicTrueSmallerCALCKeys)

In [ ]:
## golden fake and golden true

In [18]:
goldenFake = {key: dicFakeSmaller[key] for key in dicFakeSmallerGOLDKeys}
goldenTrue = {key: dicTrueSmaller[key] for key in dicTrueSmallerGOLDKeys}

In [21]:
## rest used for calculations

In [22]:
dicTrueSmallerCALC = {key: dicTrueSmaller[key] for key in dicTrueSmallerCALCKeys}

In [23]:
dicFakeSmallerCALC = {key: dicFakeSmaller[key] for key in dicFakeSmallerCALCKeys}

In [24]:
len(dicFakeSmallerCALC)

800

In [25]:
len(dicTrueSmallerCALC)

546

In [26]:
len(goldenFake)

200

In [27]:
len(goldenTrue)

136

In [28]:
len(dicTrueSmaller)

682

In [29]:
len(dicFakeSmaller)

1000

In [274]:
## trial of functions

In [275]:
WORD = re.compile(r'\w+')

In [292]:
def text_to_vector(text):
    text = relevant_words(text)
    words = WORD.findall(text)
    return Counter(words)

In [291]:
def relevant_words(text):
    blob = TextBlob(text)
    tags = blob.tags
    return " ".join([t[0] for t in tags if ((t[1] == "NN") or (t[1] == "JJ"))])

In [298]:
vec3 = text_to_vector(textFake)

In [301]:
vec2 = text_to_vector('burden')

In [302]:
intersection = set(vec3.keys()) & set(vec2.keys())

In [303]:
intersection

{'burden'}

In [314]:
import math

In [318]:
from functools import partial

In [315]:
 ## this function calculates cosine similarities between vec1 and vec2
def get_cosine(vec1, vec2):
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])

    sum1 = sum([vec1[x] ** 2 for x in vec1.keys()])
    sum2 = sum([vec2[x] ** 2 for x in vec2.keys()])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)

    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator

In [317]:
get_cosine(vec3, vec3)

1.0

In [280]:
blob = TextBlob(textFake)

In [319]:
### testing out my class

In [101]:
import re, math
import os
import textblob.tokenizers  as tt
import pickle
#from profanity import profanity

from textblob import TextBlob
from textstat.textstat import textstat as ts

from collections import Counter
from functools import partial

class NLPanalysis:

    # FIELDS

    __nameOfFinalDictionary = ''


    # dictionary in which we keep the data
    __dictionary = {}

    # you keep the result in resultingDictionary
    __resultingDictionary = {}
    __WORD = re.compile(r'\w+')

    ## golden fake is sample of fake news (HOW YOU IDENTIFIED THOSE) that you do not use in fitting the model, but using
    ## as a FEATURE base to which you calculate the (average) cosine (or other) similarities
    ## what I have done is that I extracted 20 percent of all fake to be goldenFake
    __goldenFake = {}

    __goldenFakeVector = []

    ## golden true is sample of true news (HOW YOU IDENTIFIED THOSE) that you do not use in fitting the model, but using
    ## as a FEATURE base to which you calculate the (average) cosine (or other) similarities
    ## what I have done is that I extracted 20 percent of all true to be goldenTrue
    __goldenTrue = {}

    __goldenTrueVector = []

    # CONSTRUCTOR
    ## remember put dictionary == cleaned up dictionary
    def __init__(self, goldenFake, goldenTrue, dictionary, nameOfFinalDictionaryOnDisc, fake):
        print('The dictionary you are putting in should be already in form: {text_id: text}.')

        if (type(dictionary) == dict) and (type(fake) == bool) and (type(goldenFake) == dict)\
                and (type(goldenTrue) == dict):

            self.__fake = fake
            self.__goldenFake = self.__cleanTextInFakeDictionary(goldenFake)
            self.__goldenTrue = goldenTrue

            if fake:
                print('NOTE: you are creating the fake = ' + str(fake) + ' class.')
                ## do pre-cleaning here
                self.__dictionary = self.__cleanTextInFakeDictionary(dictionary)
            else:
                print('NOTE: you are creating the fake = ' + str(fake) + ' class.')
                self.__dictionary = dictionary

            ## creating the empty dictionary here
            self.__createEmptyDictionary()

            ## finally create the name of final dictionary on disc
            self.__nameOfFinalDictionary = nameOfFinalDictionaryOnDisc + str(self.__fake)

            ## pre-compute the golden vectors
            self.__precomputeGoldenVectors()

        else:
            raise ValueError

    # HELPFUL FUNCTIONS

    ## this function pre-computes the vectors for goldenFake and goldenTrue dictionaries

    def __precomputeGoldenVectors(self):
            self.__goldenFakeVector = [self.__text_to_vector(self.__goldenFake[key]) for key in
                                       self.__goldenFake.keys()]
            self.__goldenTrueVector = [self.__text_to_vector(self.__goldenTrue[key]) for key in
                                       self.__goldenTrue.keys()]
        
            self.__saveToFile(self.__goldenFakeVector, 'goldenFakeVector')
            self.__saveToFile(self.__goldenTrueVector, 'goldenTrueVector')

    ## opening the file from the function
    def __safelyOpen(self, nameOfSavedDictionary):
        try:
            with open(nameOfSavedDictionary + '.pickle', 'rb') as handle:
                return pickle.load(handle)
        except FileNotFoundError:
            print("File was not found ")

    ## saving the structure to file
    def __saveToFile(self, structureToSave, structureNameOnDisc):
        if (structureNameOnDisc + '.pickle') in os.listdir():
            print('NOTICE')
            print('the same named file found on disc, be super that you are not rewriting it in good way')
        with open(structureNameOnDisc + '.pickle', 'wb') as handle:
            pickle.dump(structureToSave, handle)


    ## this function cleans up the fake text
    def __cleanText(self, text):
        if self.__fake:
            text = text.replace('\n', ' ')
            text = text.split('.')
            del (text[-1])
            text.append('')
            text = '.'.join(text)
            return text
        else:
            print('Nothing to do on not fake news, the text seemed clean already.')



    ## create an empty dictionary of structure dic = {article_id: []}
    def __createEmptyDictionary(self):
        self.__resultingDictionary = {key: [] for key in self.__dictionary.keys()}


    ## function calculates the properties of self.__dictionary, saves the result on disc back
    ## and adds the name of user function to funcUsed list and save it on local drive
    def __calculateFunResults(self, fun, key):
        #for key in self.__dictionary.keys():
        self.__resultingDictionary[key].extend(fun(self.__dictionary[key]))

        ### save the resultingDictionary to disc
        self.__saveToFile(self.__resultingDictionary, self.__nameOfFinalDictionary)


    ## FUNCTIONS FOR NLP. CALCULATIONS

    ## function that tokenizes the text
    def __tokenizeText(self, text):
        ### the tt.word_tokenize(text) is just the generator and is not present after you used it once, therefore,
        ### create the list out of it
        self.__tokenizedText = list(tt.word_tokenize(text))

    ## NLP FUNCTIONS:

    ## function calculates the total number of capitalized words in a text
    ## function calculates the density of capitalized words in the text
    def __checkUpperCaseWordsAndDensity(self, text):
        finalNumberOfUpper = 0
        length = float(len(self.__tokenizedText))
        for word in self.__tokenizedText:
            if word.isupper():
                finalNumberOfUpper += 1
        return [finalNumberOfUpper, finalNumberOfUpper/length]


    ## calculating readability of the text (so far only Dale-Chall readability implemented)
    def __readability_of_text(self, text, score="dale_chall"):
        try:
            if type(score) == str:
                if score == "dale_chall":
                    readability = ts.dale_chall_readability_score(text)
                    return [readability]
                else:
                    print('Other scores are not supported yet. You wanted: ' + score + " we have only dale_chall")
            else:
                raise ValueError
        except ValueError:
            print("the score should be of type str. You put " + str(type(score)))
            raise

    ## calculate number of ? and !
    ## calculate number of ? and ! density
    def __calculateQuestAndExAndDensity(self, text):
        length = float(len(self.__tokenizedText))
        qAndEx = len(list(filter(lambda x: re.match('\?|!', x), text)))
        return [qAndEx, qAndEx/length]


    ## total vulgarity of text
    ## vulgar density of the text
    def __vulgarAndDensity(self, text):
        length = float(len(self.__tokenizedText))
        prof = 0
        for word in self.__tokenizedText:
            if profanity.contains_profanity(word):
                prof += 1
        return [prof, prof/length]

    ## calculate polarity and subjectivity
    def __polarityAndSubjectivity(self, text):
        blob = TextBlob(text)
        return [blob.sentiment[0], blob.sentiment[1]]


    ## this calculates the cosine similarities of text to goldenFake and goldenTrue
    def __calculateAvgCosineSim(self, fake, text):

        if (type(fake) == bool) and fake and (type(text) == str):
            vectText = self.__text_to_vector(text)

            return [sum([self.__get_cosine(vectText, vec) for vec in self.__goldenFakeVector]) \
                   / float(len(self.__goldenFakeVector))]

        elif (type(fake) == bool) and (not fake) and (type(text) == str):
            vectText = self.__text_to_vector(text)
            return [sum([self.__get_cosine(vectText, vec) for vec in self.__goldenTrueVector])/float(len(self.__goldenTrueVector))]

        else:
            raise ValueError


    ## this function turns the text into a vector
    def __text_to_vector(self, text):
        text = self.__relevant_words(text)
        words = self.__WORD.findall(text)
        return Counter(words)

    ## this function picks up the relevant words from the text == words of type 'NN' and 'JJ' in nltk language
    def __relevant_words(self, text):
        blob = TextBlob(text)
        tags = blob.tags
        return " ".join([t[0] for t in tags if ((t[1] == "NN") or (t[1] == "JJ"))])

    ## this function calculates cosine similarities between vec1 and vec2
    def __get_cosine(self, vec1, vec2):
        intersection = set(vec1.keys()) & set(vec2.keys())
        numerator = sum([vec1[x] * vec2[x] for x in intersection])

        sum1 = sum([vec1[x] ** 2 for x in vec1.keys()])
        sum2 = sum([vec2[x] ** 2 for x in vec2.keys()])
        denominator = math.sqrt(sum1) * math.sqrt(sum2)

        if not denominator:
            return 0.0
        else:
            return float(numerator) / denominator

    # CLEANING THE FAKE TEXT
    ## function that applies clean text on whole dictionary

    def __cleanTextInFakeDictionary(self, dictionary):
        if self.__fake:
            dicFake = {new_key: self.__cleanText(dictionary[new_key]) for new_key in
                           dictionary.keys()}
            return dicFake
        else:
            print('Nothing to do on not fake news, the text seemed clean already.')
            return dictionary

    # NLP CALCULATIONS

    ## application of the list of functions on our dictionary
    def calculateNLP(self):

        ### putting the list of functions by hand, that you want to apply:
        listOfFunctions = [self.__checkUpperCaseWordsAndDensity,
                           self.__readability_of_text, self.__calculateQuestAndExAndDensity,
                           self.__vulgarAndDensity, partial(self.__calculateAvgCosineSim, True),
                           partial(self.__calculateAvgCosineSim, False)]

        ### loop through the text, for each text ---> tokenize the text ---> apply functions ---> save result to final
        ### dictionary and continue
        for key in self.__dictionary.keys():
            text = self.__dictionary[key]

            # keep the text in self.__tokenized text
            self.__tokenizeText(text)

            for fun in listOfFunctions:
                self.__calculateFunResults(fun, key)

        ### add the used function to 'usedFunctionNames' list and save it on local drive
        # self.__saveToFile([fun.__name__ for fun in listOfFunctions], 'alreadyUsedFunctionsNames' + str(self.__fake))





In [107]:
type(safelyOpen('goldenTrueVector'))

list

In [38]:
two_fake = {key: dicFakeSmallerCALC[key] for key in list(dicFakeSmallerCALC.keys())[0:2]}

In [103]:
two_nlpFake = NLPanalysis(goldenFake, goldenTrue, two_fake, 'twoTrialFakeNUMBERS', True)

The dictionary you are putting in should be already in form: {text_id: text}.
NOTE: you are creating the fake = True class.


In [51]:
two_nlpFake.calculateNLP()

NOTICE
the same named file found on disc, be super that you are not rewriting it in good way
NOTICE
the same named file found on disc, be super that you are not rewriting it in good way
NOTICE
the same named file found on disc, be super that you are not rewriting it in good way
NOTICE
the same named file found on disc, be super that you are not rewriting it in good way
NOTICE
the same named file found on disc, be super that you are not rewriting it in good way
NOTICE
the same named file found on disc, be super that you are not rewriting it in good way
NOTICE
the same named file found on disc, be super that you are not rewriting it in good way
NOTICE
the same named file found on disc, be super that you are not rewriting it in good way
NOTICE
the same named file found on disc, be super that you are not rewriting it in good way
NOTICE
the same named file found on disc, be super that you are not rewriting it in good way
NOTICE
the same named file found on disc, be super that you are not re

In [53]:
print(os.listdir())

['.DS_Store', '.idea', '.ipynb_checkpoints', '__pycache__', 'dirty_data_code_NYT.py', 'fake.csv', 'FakeData.ipynb', 'FakeData.py', 'fakeDictionaryTrialNUMBERSTrue.pickle', 'fakeKaggleEnglishOver300.pickle', 'getting_data.ipynb', 'kucapaca.pickle', 'LargerNytDataSetQuery.ipynb', 'newDict.pickle', 'NLPanalysis.ipynb', 'NLPanalysis.py', 'numericalFakeFromKaggle.pickle', 'numericalTrue2016NYT.pickle', 'nyt.csv', 'nyt.jsongetTimesArticles_testing.log', 'NytData.py', 'nytRawTestText2016.pickle', 'nytRawText2015.pickle', 'nytRawText2016.pickle', 'nytSnippets2015.pickle', 'nytSnippets2016.pickle', 'PutDataIntoSQL.ipynb', 'PutDataIntoSQL.py', 'testDic.pickle', 'TrainAndPredict.ipynb', 'TrainAndPredict.py', 'trial.pickle', 'trialFakeNUMBERSTrue.pickle', 'trialList.pickle', 'trialList1.pickle', 'trueDictionaryTrialNUMBERSFalse.pickle', 'twoTrialFakeNUMBERSTrue.pickle']


In [346]:
nlpFake = NLPanalysis(goldenFake, goldenTrue, dicFakeSmallerCALC, 'fakeDictionaryTrialNUMBERS', True)

The dictionary you are putting in should be already in form: {text_id: text}.
NOTE: you are creating the fake = True class.


In [338]:
nlpTrue = NLPanalysis(goldenFake, goldenTrue, dicTrueSmallerCALC, 'trueDictionaryTrialNUMBERS', False)

The dictionary you are putting in should be already in form: {text_id: text}.
Nothing to do on not fake news, the text seemed clean already.


In [330]:
## nlpFake calculate NLP properties

In [ ]:
nlpFake.calculateNLP()

NOTICE
the same named file found on disc, be super that you are not rewriting it in good way
NOTICE
the same named file found on disc, be super that you are not rewriting it in good way
NOTICE
the same named file found on disc, be super that you are not rewriting it in good way
NOTICE
the same named file found on disc, be super that you are not rewriting it in good way
NOTICE
the same named file found on disc, be super that you are not rewriting it in good way
NOTICE
the same named file found on disc, be super that you are not rewriting it in good way
NOTICE
the same named file found on disc, be super that you are not rewriting it in good way
NOTICE
the same named file found on disc, be super that you are not rewriting it in good way
NOTICE
the same named file found on disc, be super that you are not rewriting it in good way
NOTICE
the same named file found on disc, be super that you are not rewriting it in good way
NOTICE
the same named file found on disc, be super that you are not re

In [59]:
print(os.listdir())

['.DS_Store', '.idea', '.ipynb_checkpoints', '__pycache__', 'dirty_data_code_NYT.py', 'fake.csv', 'FakeData.ipynb', 'FakeData.py', 'fakeDictionaryTrialNUMBERSTrue.pickle', 'fakeKaggleEnglishOver300.pickle', 'getting_data.ipynb', 'kucapaca.pickle', 'LargerNytDataSetQuery.ipynb', 'newDict.pickle', 'NLPanalysis.ipynb', 'NLPanalysis.py', 'numericalFakeFromKaggle.pickle', 'numericalTrue2016NYT.pickle', 'nyt.csv', 'nyt.jsongetTimesArticles_testing.log', 'NytData.py', 'nytRawTestText2016.pickle', 'nytRawText2015.pickle', 'nytRawText2016.pickle', 'nytSnippets2015.pickle', 'nytSnippets2016.pickle', 'PutDataIntoSQL.ipynb', 'PutDataIntoSQL.py', 'testDic.pickle', 'TrainAndPredict.ipynb', 'TrainAndPredict.py', 'trial.pickle', 'trialFakeNUMBERSTrue.pickle', 'trialList.pickle', 'trialList1.pickle', 'trueDictionaryTrialNUMBERSFalse.pickle', 'twoTrialFakeNUMBERSTrue.pickle']


In [ ]:
## Fitting the model

In [87]:
numericFake = safelyOpen('fakeDictionaryTrialNUMBERSTrue')

In [61]:
numericTrue = safelyOpen('trueDictionaryTrialNUMBERSFalse')

In [70]:
dfFake = pd.DataFrame(list(numericFake.values()))

In [71]:
dfTrue = pd.DataFrame(list(numericTrue.values()))

In [130]:
## PIPELINE PIPELINE PIPELINE, implement into class text_to_model

In [125]:
dataFake = safelyOpen('fakeKaggleEnglishOver300')

In [129]:
dicFake =  createDictionary(dataFake, 'text')

In [120]:
dataTrue2015 = safelyOpen('nytRawText2015')
dataTrue2016 = safelyOpen('nytRawText2016')

In [121]:
len(dataTrue2015)

647

In [122]:
len(dataTrue2016)

682

In [126]:
len(dataFake)

7409

In [139]:
lel = safelyOpen('fakeKaggleEnglishOver300')

In [141]:
createDictionary(lel, 'site_url')

{'0206b54719c7e241ffe0ad4315b808290dbe6c0f': '100percentfedup.com',
 'd3cc0fe38f41a59f7c48f8c3528ca5f74193148f': '100percentfedup.com',
 'b4bbf8b5c19e8864f5257832a58b81ef4ed2d4e4': '100percentfedup.com',
 '7f90d71cc69893f4c91e8a55f2c4da52d88cd7c1': '100percentfedup.com',
 'fd2c048d7e03f7260a8d75725b0441c627578f1a': '100percentfedup.com',
 'c8d4c0a88af223f1b41e3e966ecd98ceb22ab5fb': '100percentfedup.com',
 '162b39e6518af4e591acdd65d719b4f858596b3e': '100percentfedup.com',
 '1fe4c0fa4ba9e410fe5c4e7fec13c6740062629b': '100percentfedup.com',
 '3836cd38353373b7d934a7e25e5888876958f94a': '100percentfedup.com',
 'c1a86b752a21196a06591d5a785fa29b14fab245': '100percentfedup.com',
 'aff3ede5c01b6823e3bb6f96035c4df02d470d98': '21stcenturywire.com',
 '7759496d1762e34d0a33db572948fd77df481d1d': '21stcenturywire.com',
 'f51a0aa7d160a15707b18a4007bdfa4c506d17a6': '21stcenturywire.com',
 '236eaaea138082b5f4e8362de5e8e5584d8e7ef4': '21stcenturywire.com',
 '9705f82da90935a714b424d8fafa93fba75be998': '21